In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [16]:
def order_type_distribution(df):
    order_type = df['type']

    # PIE chart
    type_flatten = order_type.value_counts()[::-1]
    plt.figure(figsize=(5, 5))
    plt.pie(type_flatten, labels=type_flatten.index, autopct='%1.1f%%', colors=plt.cm.tab20.colors)
    plt.title('Order Type distribution')
    plt.savefig('./Type Frequency.png')
    plt.close()

def latency_histogram(df):
    nanosecond = df['nanoseconds']
    average_time = nanosecond.mean()

    # Histogram
    cut = nanosecond[nanosecond <= nanosecond.quantile(q=0.99)]
    plt.figure(figsize=(10, 6))
    plt.hist(cut, bins=30, color='skyblue', edgecolor='black')
    plt.title(f'Histogram of Latency - mean ={average_time:.1f}ns at ({1e9/average_time:,.0f} orders/s)')
    plt.xlabel('Latency (ns)')
    plt.ylabel('Number of Orders')
    plt.savefig('./Latency Histogram.png')
    plt.close()
    
def order_type_latency(df):
    # Calculate stats
    stats = df.groupby('type')['nanoseconds'].agg(['mean', lambda x: x.quantile(0.15), lambda x: x.quantile(0.85)]).reset_index()
    stats.columns = ['type', 'mean', '25per', '75per']
    stats['l'] = stats['mean'] - stats['25per']
    stats['u'] = stats['75per'] - stats['mean']
    stats = stats.sort_values(by='mean')

    # Create box and whisker
    plt.figure(figsize=(12, 6))
    plt.bar(stats['type'], stats['mean'], yerr=[stats['l'], stats['u']], capsize=5, color='skyblue', edgecolor='black')
    plt.title('Latency by Order Type')
    plt.xlabel('Type')
    plt.ylabel('Latency (ns)')
    plt.xticks(rotation=45)
    plt.savefig('./Order Type Latency.png', bbox_inches='tight')
    plt.close()

In [17]:

input_file = '../data/performance.txt'
df = pd.read_csv(input_file, header=None, names=['type', 'nanoseconds', 'executed order count', 'AVL balance count'])
order_type_distribution(df)
latency_histogram(df)
order_type_latency(df)